In [30]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D, Dense
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import cv2
import numpy as np


dataset = 'data/fer2013.csv'
image_size = (48,48)

def load_data():
    data = pd.read_csv(dataset)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'),image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()
    return faces, emotions

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

faces, emotions = load_data()
faces = preprocess_input(faces)
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [38]:
# parameters
batch_size = 32
num_epochs = 50
input_shape = (48, 48, 1)
verbose = 1
num_classes = 7
patience = 50
base_path = 'models/'
l2_regularization=0.01
 
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 44, 44, 32)        832       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 18, 18, 64)        51264     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 9, 9, 64)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 5184)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1000)              5185000   
_________________________________________________________________
dense_10 (Dense)             (None, 7)                 7007      
Total para

In [39]:
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = base_path + '_CNN_model_'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]


faces, emotions = load_data()
faces = preprocess_input(faces)
num_samples, num_classes = emotions.shape
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)
model.fit_generator(data_generator.flow(xtrain, ytrain,
                                            batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Epoch 1/50
898/897 [==============================] - 16s 18ms/step - loss: 1.6446 - acc: 0.3435 - val_loss: 1.4189 - val_acc: 0.4532

Epoch 00001: val_loss improved from inf to 1.41890, saving model to models/_CNN_model_.01-0.45.hdf5
Epoch 2/50
898/897 [==============================] - 15s 17ms/step - loss: 1.4434 - acc: 0.4421 - val_loss: 1.3262 - val_acc: 0.4939

Epoch 00002: val_loss improved from 1.41890 to 1.32625, saving model to models/_CNN_model_.02-0.49.hdf5
Epoch 3/50
898/897 [==============================] - 15s 17ms/step - loss: 1.3532 - acc: 0.4786 - val_loss: 1.2421 - val_acc: 0.5259

Epoch 00003: val_loss improved from 1.32625 to 1.24209, saving model to models/_CNN_model_.03-0.53.hdf5
Epoch 4/50
898/897 [==============================] - 15s 17ms/step - loss: 1.2995 - acc: 0.5023 - val_loss: 1.2268 - val_acc: 0.5373

Epoch 00004: val_loss improved from 1.24209 to 1.22678, saving model to models/_CNN_model_.04-0.54.hdf5
Epoch 5/50
898/897 [============================

In [45]:
import cv2
import numpy as np
from keras.models import load_model
faceCascade = cv2.CascadeClassifier('haarcascade_files/haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)
model = load_model('models/_CNN_model_.46-0.63.hdf5')

target = ['angry','disgust','fear','happy','sad','surprise','neutral']
font = cv2.FONT_HERSHEY_SIMPLEX
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1)

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2,5)
        face_crop = frame[y:y+h,x:x+w]
        face_crop = cv2.resize(face_crop,(48,48))
        face_crop = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
        face_crop = face_crop.astype('float32')/255
        face_crop = np.asarray(face_crop)
        face_crop = face_crop.reshape(1, face_crop.shape[0],face_crop.shape[1],1)
        result = target[np.argmax(model.predict(face_crop))]
        cv2.putText(frame,result,(x,y), font, 1, (255,255,255), 3, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('s'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [61]:
import pandas as pd

log_data  = open('models/_emotion_training.log', 'r')
split_list = []

for line in log_data:
    data = line.split(',')
    #print(thing1)
    epochs = data[0]
    acc = data[1]
    loss = data[2]
    val_acc = data[3]
    val_loss = data[4]

    split_list.append([epochs, acc, loss, val_acc, val_loss])

df = pd.DataFrame(split_list, columns=['epochs','acc','loss','val_acc','val_loss'])
df

,epochs,acc,loss,val_acc,val_loss
0,epoch,acc,loss,val_acc,val_loss\n
1,0,0.34658121146730647,1.6297167692693026,0.4600167283422437,1.4280669269525397\n
2,1,0.450207252082267,1.4145038691329053,0.5114238062490989,1.2731193671125984\n
3,2,0.48880142115764746,1.3314379798178015,0.5339927676827438,1.2302504519120536\n
4,3,0.5095266292805357,1.2820175922237937,0.5231262306527329,1.2427245141836087\n
5,4,0.5243651816524297,1.248310371660888,0.5493173708230481,1.185818754108537\n
6,5,0.5337002333804455,1.2211829432042989,0.5647812887972263,1.1619964280926307\n
7,6,0.5410846772809782,1.2030045355249503,0.5565617286493002,1.1619291304753132\n
8,7,0.5513950329175451,1.175949544761949,0.5697966132984582,1.130658319904491\n
9,8,0.5582569925812428,1.1650808358243103,0.5770409709378749,1.1171059515000252\n
